In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [2]:
#Step 1: Data Extraction and Loading

#Extract Data: Load the necessary tables from the Ergast dataset, which includes drivers, constructors, races, qualifying, results, driverStandings, and constructorStandings.
#Load Data: Use a database client (e.g., MySQL, SQLite) or CSV files to load the data into a pandas DataFrame.
# Load data
constructors_df = pd.read_csv("f1db_csv/constructors.csv")
races_df=pd.read_csv("f1db_csv/races.csv")
quali_df=pd.read_csv("f1db_csv/qualifying.csv")
results_df=pd.read_csv("f1db_csv/results.csv")
driverStandings_df=pd.read_csv("f1db_csv/driver_standings.csv")
constructorStandings_df=pd.read_csv("f1db_csv/constructor_standings.csv")
drivers_df = pd.read_csv('f1db_csv/drivers.csv')

In [3]:
#Step 2: Data Preprocessing

#Merge Tables:
#Merge results with races to get race-specific details.
#Merge the combined DataFrame with qualifying to get qualifying positions.
#Merge with driverStandings and constructorStandings to include standings.
#Merge with drivers and constructors to add driver and constructor details.

# Merge data
data = results_df.merge(races_df, on="raceId")
data1 = data.merge(quali_df, on=["raceId","driverId","constructorId"], how="left")
data2 = data1.merge(driverStandings_df, on=["raceId","driverId"], how="left")
data3 = data2.merge(constructorStandings_df, on=['raceId','constructorId'])
data4 = data3.merge(drivers_df, on=['driverId'], how='left')
data5 = data4.merge(constructors_df, on=['constructorId'], how='left')

In [5]:
#Handle Missing Values:
#Fill or drop missing values as appropriate for the analysis.
#Common strategies include filling with mean/median/mode or dropping rows with missing critical values.
#Convert Data Types:
#Ensure all columns are in the appropriate data type (e.g., integers for IDs, floats for numerical values).

columns=data5.columns
data5.replace('\\N', 0, inplace=True)
data5.tail()
total_n_values = (data5.applymap(lambda x: x == '\\N')).sum().sum()
print(total_n_values)

data5.to_csv('output.csv', index=False)
f1data=pd.read_csv("output.csv", low_memory=False)
f1data.drop(['drivers_url','const_url'], axis=1, inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_8764\2383923519.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  total_n_values = (data5.applymap(lambda x: x == '\\N')).sum().sum()


0


In [7]:
f1data.head()

,resultId,raceId,driverId,constructorId,results_number,results_grid,results_position,results_positionText,results_positionOrder,results_points,...,driverRef,drivers_number,drivers_code,drivers_forename,drivers_surname,drivers_dob,drivers_nationality,constructorRef,const_name,const_nationality
0,1,18,1,1,22,1,1,1,1,10.0,...,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,mclaren,McLaren,British
1,2,18,2,2,3,5,2,2,2,8.0,...,heidfeld,0,HEI,Nick,Heidfeld,1977-05-10,German,bmw_sauber,BMW Sauber,German
2,3,18,3,3,7,7,3,3,3,6.0,...,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,williams,Williams,British
3,4,18,4,4,5,11,4,4,4,5.0,...,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,renault,Renault,French
4,5,18,5,1,23,3,5,5,5,4.0,...,kovalainen,0,KOV,Heikki,Kovalainen,1981-10-19,Finnish,mclaren,McLaren,British


In [8]:
#Step 3: Feature Engineering

#Create New Features:
#Qualifying Position: Use the qualifying position as a feature.
#Previous Race Results: Calculate the average finishing position of the driver and constructor in previous races.
#Circuit Characteristics: Extract features such as circuit length, number of turns, and altitude.
#Weather Conditions: If available, include weather conditions like temperature, rain probability, etc.
#Driver and Constructor Standings: Use the current standings points and positions.
#Normalize/Standardize Features: Scale numerical features to ensure they have a mean of 0 and standard deviation of 1 (standardization) or scale between 0 and 1 (normalization).

# Feature engineering
f1data['average_finish'] = f1data.groupby('driverId')['results_positionOrder'].transform('mean')

# Select features and target
features = ['results_grid', 'quali_position', 'average_finish', 'driver_standings_points', 'driver_standings_position', 'const_standing_points', 'const_standing_position']
target = 'results_positionOrder'
X = f1data[features]
y = f1data[target]

In [9]:
#Step 4: Data Splitting

#Split the Data:
#Split the data into training and test sets (e.g., 80% training, 20% testing).
#Optionally, create a validation set from the training set for hyperparameter tuning.

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [10]:
#Step 5: Model Selection and Training
#Choose a Model:
#Regression models like Linear Regression, Ridge Regression.
#Tree-based models like Decision Trees, Random Forests, Gradient Boosting.
#Ensemble models like XGBoost, LightGBM.

#Train the Model:
#Train the chosen model using the training set.
#Use cross-validation to tune hyperparameters and avoid overfitting.

# Train model
model = RandomForestRegressor()
model.fit(X_train, y_train)

RandomForestRegressor()

In [11]:
#Step 6: Model Evaluation
#Evaluate the Model:
#Use the test set to evaluate model performance.
#Metrics to use include Mean Absolute Error (MAE), Mean Squared Error (MSE), Root Mean Squared Error (RMSE).

#Feature Importance:
#Analyze feature importance to understand which features contribute the most to the prediction.

# Predict and evaluate
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 4.404995835321738


In [12]:
#Step 7: Model Tuning and Optimization

#Hyperparameter Tuning:
#Use Grid Search or Random Search for hyperparameter tuning.
#Optimize parameters like the number of trees, depth of trees, learning rate, etc.

#Ensemble Methods:
#Combine multiple models to improve prediction accuracy (e.g., stacking, bagging).

# Hyperparameter tuning (optional)
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30]
}
grid_search = GridSearchCV(model, param_grid, cv=3, scoring='neg_mean_absolute_error')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [10, 20, 30],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_absolute_error')

In [17]:
#Step 8: Deployment and Monitoring

#Deploy the Model:
#Save the trained model using joblib or pickle.
#Deploy the model to a production environment using a web framework like Flask or FastAPI.

#Monitor the Model:
#Continuously monitor model performance and update it with new data as necessary.

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Best Model Mean Absolute Error: {mae}')

Best Model Mean Absolute Error: 4.229701889150435
